# Relatório de Validação da Tabela STG_PGT

# Importando as bibliotecas, definindo as funções e carregando as tabelas necessárias

In [1]:
#Importa as bibliotecas necessárias
import pandas as pd
import numpy as np
import datetime
from plotly.offline import iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import os

In [2]:
#Cria as pastas necessárias
pasta ="./relatorio/pags"

access_rights = 0o777

try:
    os.mkdir('relatorio', access_rights)
except OSError:
    print ("\tAVISO: A criação da pasta (relatorio) falhou ou a pasta já existe")
else:
    print ("\tPasta (relatorio) criada com sucesso!")

try:
    os.mkdir('relatorio/pags', access_rights)
except OSError:
    print ("\tAVISO: A criação da pasta (pags) falhou ou a pasta já existe")
else:
    print ("\tPasta (pags) criada com sucesso!")

	AVISO: A criação da pasta (relatorio) falhou ou a pasta já existe
	AVISO: A criação da pasta (pags) falhou ou a pasta já existe


In [3]:
#Define as funções personalizadas necessárias
def write_to_html_file(df, title='', filename='out.html'):
    result = '''
<html>
<head>
<style>

    h2 {
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
    }
    body{ 
        display: flex;
        align-items: center;
        justify-content: center;
    }
    table { 
        margin-left: auto;
        margin-right: auto;
        width: 90%;
        color: #707070;
    }
    table, th, td {
        border: 1px solid #707070;
        border-collapse: collapse;
    }
    th {
        padding: 5px;
        text-align: left;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    td {
        padding: 5px;
        text-align: right;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    table tbody tr:hover {
        background-color: #dddddd;
    }
    .wide {
        width: 90%; 
    }

</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    if type(df) == pd.io.formats.style.Styler:
        result += df.render()
    else:
        result += df.to_html(classes='wide', escape=False)
    result += '''
</body>
</html>
'''
    with open(filename, 'w') as f:
        f.write(result)

def highlight_max(s):    
    is_max = s == s.max()
    return ['color: white; background-color: #3749E9' if v else '' for v in is_max]

def highlight_min(s):
    is_max = s == s.min()
    return ['color: white; background-color: #112244' if v else '' for v in is_max]

def destaque(val):
    color = '#F2F200' if val == 'Sim' else ''
    return 'background: {}'.format(color)

def destaque_coluna(val):
    color = '#F2F200' if val == 0 else ''
    return 'background: {}'.format(color)

In [4]:
#Carregas os dataframes
df = pd.read_excel ('STG_PGT.xlsx')
cf = pd.read_excel ('STG_MDL.xlsx')
bf = pd.read_excel ('STG_FNT_ITT.xlsx')

In [5]:
#df

In [6]:
#df.dtypes

# Verificando os tipos das colunas

In [7]:
#Trata as datas
df.DAT_VCT = pd.to_datetime(df.DAT_VCT, format='%d%m%Y', errors='coerce')
df.DAT_RSS_FNT_ITT = pd.to_datetime(df.DAT_RSS_FNT_ITT, format='%Y-%m-%d', errors='coerce')
df.DAT_INC_DBO = pd.to_datetime(df.DAT_INC_DBO, format='%Y-%m-%d', errors='coerce')

#Verifica as linhas das colunas o tipo desejado
dict_tipo = {'ID_STG_PGT': int, 'VLR_PGT_FAT': int, 'DAT_VCT': datetime.date, 'COD_MDL': str,
             'QTD_CLI_CAD_POS': int, 'QTD_PGT': int, 'ID_FNT_ITT': int, 'DES_TIP_PSS': ('F' or 'J'),
             'DAT_RSS_FNT_ITT': datetime.date, 'DAT_INC_DBO': datetime.date}
dict_analise = {}
for coluna, tipo in dict_tipo.items():
    dict_analise[coluna] = len([linha for linha in df[coluna]
                                if (type(linha) == tipo) or (linha == tipo) or (isinstance(linha, tipo))])

#Calcula a porcentagem do preenchimento por coluna
final = [x * 100 / len (df) for x in dict_analise.values()]

#Cria um dataframe
tipo_review = pd.DataFrame({'Porcentagem': final},
                           index=df.columns)
tipo_review = tipo_review.rename_axis('Colunas', axis='columns')
#tipo_review

In [8]:
#Plota em formato gráfico e exporta para HTML
data = [go.Bar(x = final,
               y = df.columns,
               orientation = 'h',
               marker=dict(color='#112244')
              )]

fig = go.Figure(data=data)
fig.update_yaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(xaxis=dict(ticksuffix = '%'))
iplot(fig)
fig.write_html(pasta + "/pgt_tipo.html")

# Define quais colunas possuem dados sensíveis

In [9]:
#Cria um dataframe definindo as colunas com dados sensíveis
dado_sensivel = ['Não', 'Não', 'Não', 'Não', 'Não', 'Não', 'Sim', 'Não', 'Não', 'Não']

sensibilidade_review = pd.DataFrame({'Dado Sensível': dado_sensivel},
                                      index=df.columns)
sensibilidade_review = sensibilidade_review.rename_axis('Colunas', axis='columns')

visualiza = (sensibilidade_review
                .style
                .applymap(destaque))
write_to_html_file(visualiza, title='', filename=pasta + "/pgt_tab_sensibilidade.html")
visualiza

# Verifica as informações com outras tabelas

In [10]:
#Compara os valores da Tabela PGT com a Tabela FNT
fontes = df.groupby('ID_FNT_ITT').count().index
modalidades = df.groupby('COD_MDL').count().index

verificabf = []
for x in bf.ID_STG_FNT_ITT:
    verificabf.append (x)

diferente = 0
for x in df.ID_FNT_ITT:
    if x not in verificabf:
        diferente += 1

if diferente == 0:
    porcentagem_FNT = 100
else:
    porcentagem_FNT = 100 - (diferente *100/len(df.ID_FNT_ITT))

verificacf = []
for x in cf.COD_MDL:
    verificacf.append (x)

diferente = 0
for x in df.COD_MDL:
    if x not in verificacf:
        diferente += 1

#Faz o cálculo de porcentagem baseado no valor total de linhas
if diferente == 0:
    porcentagem_COD = 100
else:
    porcentagem_COD = 100 - (diferente *100/len(df.COD_MDL))

#Cria um dataframe
porcentagens = [porcentagem_FNT, porcentagem_COD]
existencia_labels = ['IDs de Fontes', 'Códigos de Modalidades']
encontrados = [len(fontes), len(modalidades)]

correspondencia_review = pd.DataFrame({'Encontrados': encontrados,
                                       'Correspondência': porcentagens},
                                      index=existencia_labels)
correspondencia_review = correspondencia_review.rename_axis('Colunas', axis='columns')

visualiza = (correspondencia_review
                .style
                .format({'Correspondência':"{:.2f}%"})
                .applymap(destaque_coluna, subset=pd.IndexSlice[:, ['Correspondência']]))
write_to_html_file(visualiza, title='', filename=pasta + "/pgt_tab_correspondencia.html")
visualiza

# Analisa quantos campos não são duplicados

In [11]:
#Analisa o número de valores únicos e que não foram duplicados
unico = [len(np.unique(df.ID_STG_PGT)) * 100 / len (df)]
ff = df[['VLR_PGT_FAT', 'DAT_VCT', 'COD_MDL', 'QTD_CLI_CAD_POS', 'QTD_PGT', 'ID_FNT_ITT', 'DES_TIP_PSS',
         'DAT_RSS_FNT_ITT', 'DAT_INC_DBO']]

for x in ff.columns:
    coluna = df
    coluna = coluna.replace(0,np.nan)
    soma = coluna.groupby(f'{x}').count()
    soma = soma[soma.ID_STG_PGT == 1]
    soma = soma.ID_STG_PGT.sum()
    unico.append (soma * 100 / len (coluna[x]))

#Cria um dataframe
unico_review = pd.DataFrame({'Campos Não Duplicados': unico},
                           index=df.columns)
unico_review = unico_review.rename_axis('Colunas', axis='columns')
#unico_review

In [12]:
#Plota em formato gráfico e exporta em HTML
data = go.Bar(y = unico, 
              x = df.columns, 
              marker = {'color' : '#00B7CC'})

layout = go.Layout(title = '', 
                   yaxis = {'title': ''}, 
                   xaxis = {'title': ''})

fig = go.Figure(data = data, layout = layout)
fig.update_xaxes(showline = True, linewidth = 1, linecolor = '#717171')
fig.update_yaxes(showgrid = True, gridwidth = 1, gridcolor = '#D9D9DE')
fig.update_layout({'plot_bgcolor': '#FFFFFF', 'paper_bgcolor': '#FFFFFF'})
fig.update_layout(yaxis=dict(ticksuffix = '%'))
iplot(fig)
fig.write_html(pasta + "/pgt_unicos.html")

# Analisando o preenchimento das colunas

In [13]:
#df.dtypes

In [14]:
#Completa espaços vazios com zero
df = df.loc[:, df.columns != ('DAT_VCT', 'DAT_RSS_FNT_ITT', 'DAT_INC_DBO')]
df.fillna(0)

#Converte as colunas para os valores corretos
df.ID_STG_PGT = df.ID_STG_PGT.astype (np.int64)
df.VLR_PGT_FAT = df.VLR_PGT_FAT.astype (np.int64)
df.COD_MDL = df.COD_MDL.astype(str)
df.QTD_CLI_CAD_POS = df.QTD_CLI_CAD_POS.astype(np.int64)
df.QTD_PGT = df.QTD_PGT.astype(np.int64)
df.ID_FNT_ITT = df.ID_FNT_ITT.astype (np.int64)
df.DES_TIP_PSS = df.DES_TIP_PSS.astype (str)
#df.dtypes

In [15]:
#Completa espaços vazios com zero e exclui as linhas com valor zero
analisar = df.fillna(0)
analisar = analisar != 0
analisar = analisar.sum()
analisar

#Calcula a porcentagem de preenchimento das colunas
situacao = {x[0] : (x[1] * 100 / len (df)) for x in analisar.items()}

#Cria um dataframe
tabela_review = pd.DataFrame({'COMPLETUDE (%)' : [x for x in situacao.values()]},
                             index = [x for x in situacao.keys()])
tabela_review = tabela_review.rename_axis('Colunas', axis='columns')
tabela_review['COMPLETUDE (%)'][2] = len(df.DAT_VCT) * 100 / len (df)
#tabela_review

In [16]:
#Plota em formato gráfico e exporta para HTML
data = [go.Bar(x=tabela_review['COMPLETUDE (%)'],
               y=[x for x in situacao.keys()],
               orientation = 'h',
               marker=dict(color='#3749E9')
              )]

fig = go.Figure(data=data)
fig.update_yaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(xaxis=dict(ticksuffix = '%'))
iplot(fig)
fig.write_html(pasta + "/pgt_preenchimento.html")

# Contando número de datas diferentes

In [17]:
#Separa por grupos os valores diferentes encontrados nas colunas de datas
datas_inc = df[df['DAT_INC_DBO'] != 0]
datas_inc = datas_inc.groupby('DAT_INC_DBO').count()
datas_inc

datas_rss = df[df['DAT_RSS_FNT_ITT'] != 0]
datas_rss = datas_rss.groupby('DAT_RSS_FNT_ITT').count()
datas_rss

#Analisa os valores únicos encontrados e calcula a porcentagem de seu tamanho com o número de linhas da tabela
conta = 0
porcentagem = []

for x in datas_inc.ID_STG_PGT:
    conta = x * 100 / len(df)
    porcentagem.append (conta)

for x in datas_rss.ID_STG_PGT:
    conta = x * 100 / len(df)
    porcentagem.append (conta)

#Concatena e cria uma tabela com todos os resultados
dados = [datas_inc, datas_rss]
datas_encontradas = pd.concat(dados)
datas_encontradas = pd.DataFrame({'Porcentagem': porcentagem},
                           index=datas_encontradas.index)
datas_encontradas = datas_encontradas.rename_axis('Datas Encontradas', axis='columns')
datas_encontradas.index.name = None

visualiza = (datas_encontradas
                .style
                .format({'Porcentagem':"{:.2f}%"})
                .applymap(destaque_coluna, subset=pd.IndexSlice[:, ['Porcentagem']]))
write_to_html_file(visualiza, title='', filename=pasta + "/pgt_tab_datas.html")
visualiza

# Indicadores de Validação

In [18]:
#Calcula a média de preenchimento da remessa
media_preenchimento = sum (tabela_review['COMPLETUDE (%)']) / df.shape[1]

#Calcula de validação  
media_validacao = (sum (final) /df.shape[1] + sum (porcentagens) / len (porcentagens)  + sum (porcentagem) / len (porcentagem) ) / 3

#media_preenchimento, media_validacao